In [1]:
# Library Imports

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
# read list of tickers in SP 500 

stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
# pass in API token for IEX_Cloud sandbox 
# Api Docs: https://iexcloud.io/docs/api/

from secrets import IEX_CLOUD_API_TOKEN

In [4]:
# test API call on AAPL

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 86138851, 'calculationPrice': 'close', 'change': 1, 'changePercent': 0.00702, 'close': 150.66, 'closeSource': 'faicolfi', 'closeTime': 1684794288599, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 149.25, 'delayedPriceTime': 1666039973860, 'extendedChange': 0.07, 'extendedChangePercent': 0.0005, 'extendedPrice': 153.6, 'extendedPriceTime': 1713262549440, 'high': 152.54, 'highSource': 'meynpelaict1d idreu5 e ', 'highTime': 1660364784744, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 150.14, 'iexCloseTime': 1650726915369, 'iexLastUpdated': 1711149313104, 'iexMarketPercent': 0.0114921931751001, 'iexOpen': 152.24, 'iexOpenTime': 1683157472279, 'iexRealtimePrice': 147.18, 'iexRealtimeSize': 5, 'iexVolume': 740063, 'lastTradeTime': 1699858462362, 'latestPrice': 152.5, 'latestSource': 'Close', 'latestTime': 'September 23, 2021', 'latestUpdate': 1711447052845, 'latestVolume': 65355469, 'low': 150.57, 'lowSource': 'pnmeilare

In [5]:
# Parsing Our API Call

price = data["latestPrice"]
market_cap = data['marketCap']
print(price)
print(market_cap)

152.5
2430166640348


In [6]:
# Adding Our Stocks Data to a Pandas DataFrame

my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
# Add the data series to popualte rows 

final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,152.5,2430166640348,N/A


In [8]:
# Looping Through The Tickers in Our List of Stocks

final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns), 
    ignore_index = True
    )

In [9]:
# Call final_dataframe for first 5 stocks to test

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,175.88,53503352699,N/A
1,AAL,21.50,14044418006,N/A
2,AAP,215.37,13811911723,N/A
3,AAPL,149.78,2443884371864,N/A
4,ABBV,110.16,193843177719,N/A


In [10]:
# Batch API calls to improve performance
# IEX limits batch API to group of 100 calls 

def chunks(lst, n):
    '''Yield successive n-sized chunks form lst.'''
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [11]:
# batching returns data on all 500 tickers much faster 

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings)
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
                ],
                index = my_columns),
                ignore_index = True
            )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,174.60,53864969180,N/A
1,AAL,21.40,13544669087,N/A
2,AAP,215.16,13822769191,N/A
3,AAPL,153.44,2448014649055,N/A
4,ABBV,110.91,190167391143,N/A
...,...,...,...,...
500,YUM,127.74,38609388829,N/A
501,ZBH,155.60,31819151191,N/A
502,ZBRA,576.06,30574645622,N/A
503,ZION,62.15,9689584297,N/A


In [12]:
# Calculating the Number of Shares to Buy
# Notice enter a non-int twice will break the try-except, patch to come

portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number!\nPlease try aagin: ")
    portfolio_size = input('Enter the value of your portfolio: ')

10000000.0


In [13]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,Number Of Shares to Buy
0,A,174.60,53864969180,N/A,113.0
1,AAL,21.40,13544669087,N/A,925.0
2,AAP,215.16,13822769191,N/A,92.0
3,AAPL,153.44,2448014649055,N/A,129.0
4,ABBV,110.91,190167391143,N/A,178.0
...,...,...,...,...,...
500,YUM,127.74,38609388829,N/A,155.0
501,ZBH,155.60,31819151191,N/A,127.0
502,ZBRA,576.06,30574645622,N/A,34.0
503,ZION,62.15,9689584297,N/A,318.0


In [14]:
# Initializing our XlsxWriter Object

writer = pd.ExcelWriter('Recommended Trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [15]:
## Formats for .xlsx output 
# String format for tickers
# \\$XX.XX format for stock prices
# \\$XX,XXX format for market capitalization
# Integer format for the number of shares to purchase

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color, 
        'border': 1,
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color, 
        'border': 1,
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color, 
        'border': 1,
    }
)

In [18]:
# Define column formats, loop over all rows 

column_formats = {
    "A": ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    "C": ['Market Capitalization', dollar_format],
    "D": ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [20]:
# Save output

writer.save()